# Causal Knowledge Transfer for Safe Reinforcement Learning

### Train Agents
#### Sumo Network File
When editing the sumo network (nets/simple_unprotected_right.net.xml) never edit the xml directly. Instead, go to nets/netconfig and make desired changes there. Generate the new net.xml by executing generate config.sh
#### Sumo Route File
This is part of the generate_config.sh now.

**Desired Output: Trained_Model.zip**

### Creating the Sumo Environment

In [ ]:
import json
from pathlib import Path
import xml.etree.ElementTree as ET

# Environment Variables (Ensure SUMO_HOME is properly set)
sumo_home = %env SUMO_HOME

# Constants / Parameters
SPEED = 22.22
FRICTION = 1.0
INSERT_PROBABILITY = 0.1
DURATION = 3600
REPEAT_PERIOD = 10
DEFAULT_DECEL = 4.5
DEFAULT_EMERGENCY_DECEL = 9.0

# File Paths
config_directory = Path().joinpath('nets', '2lane_unprotected_right')
config_files = {
    'netccfg_edges': config_directory.joinpath('netconfig', 'edges.edg.xml'),
    'netccfg': config_directory.joinpath('2lane_unprotected_right.netccfg'),
    'duarcfg': config_directory.joinpath('2lane_unprotected_right.duarcfg'),
    'net.xml': config_directory.joinpath('2lane_unprotected_right.net.xml'),
    'rou.xml': config_directory.joinpath('2lane_unprotected_right.rou.xml'),
    'routes.rou.xml': config_directory.joinpath('routes.rou.xml'),
    'config.rou.xml': config_directory.joinpath('config.rou.xml'),
    'experimental.rou.xml': config_directory.joinpath('experimental.rou.xml'),
}
findAllRoutes = Path(sumo_home).joinpath('tools', 'findAllRoutes.py')
vehicle2flow = Path(sumo_home).joinpath('tools', 'route', 'vehicle2flow.py')


# Update Friction Coefficients in Edge Configuration
def update_friction_coefficients(file_path, friction):
    edges_xml_tree = ET.parse(file_path)
    edges_xml_root = edges_xml_tree.getroot()
    for param in edges_xml_root.findall(".//lane/param[@key='frictionCoefficient']"):
        param.set('value', str(friction))
    edges_xml_tree.write(file_path)


update_friction_coefficients(config_files['netccfg_edges'], FRICTION)

# Execute SUMO Tools
! netconvert --configuration-file {config_files['netccfg']}
! python {findAllRoutes} -n {config_files['net.xml']} -o {config_files['routes.rou.xml']} -s southJunction,westJunction -t junctionEast,junctionNorth
! duarouter --configuration-file {config_files['duarcfg']}
! python {vehicle2flow} {config_files['config.rou.xml']} -o {config_files['rou.xml']} -e {DURATION} -r {REPEAT_PERIOD}

# Update Vehicle Configuration for Friction Adjusted Braking Distance
def update_vehicle_type_parameters(file_path, speed, default_decel, default_emergency_decel, friction):
    tree = ET.parse(file_path)
    root = tree.getroot()
    for vType in root.findall('vType'):
        vClass = vType.attrib.get('vClass')
        if vClass and vClass != 'passenger':
            raise NotImplementedError("Check for non-passenger vehicle classes not implemented")
        vType.attrib.update({
            'maxSpeed': str(speed),
            'decel': str(default_decel * friction),
            'emergencyDecel': str(default_emergency_decel * friction),
        })
    tree.write(file_path, xml_declaration=True, encoding='UTF-8')


update_vehicle_type_parameters(config_files['rou.xml'], SPEED, DEFAULT_DECEL, DEFAULT_EMERGENCY_DECEL, FRICTION)


# Update Vehicle Flows for Forcing Unprotected Right Action
def update_flows(file_path, insert_probability):
    tree = ET.parse(file_path)
    root = tree.getroot()
    for flow in root.findall('flow'):
        match flow.attrib.get('id'):
            case 'southEast':
                flow.set('period', f"exp({insert_probability})")
            case 'southNorth':
                flow.set('period', f"exp({insert_probability})")
            case 'westEastTop':
                flow.set('period', f"exp({2 * insert_probability})")
            case id if 'westEastBottom' in id:
                flow.set('end', str(float(flow.get('begin')) + 600))
                if float(flow.get('begin')) % 1200 == 0:
                    flow.set('period', f"exp({1 * insert_probability})")
                else:
                    flow.set('period', f"exp({0.0001 * insert_probability})")
    tree.write(file_path, xml_declaration=True, encoding='UTF-8')


update_flows(config_files['rou.xml'], INSERT_PROBABILITY)

In [ ]:
from env.SumoEnvironmentGenerator import SumoEnvironmentGenerator
from pathlib import Path

net_name = '2lane_unprotected_right'

environments = SumoEnvironmentGenerator(
    net_file=str(Path().joinpath('nets', net_name, f'{net_name}.net.xml')),
    route_file=str(Path().joinpath('nets', net_name, f'{net_name}.rou.xml')),
    sumocfg_file=str(Path().joinpath('nets', net_name, f'{net_name}.sumocfg')),
    duration=3600,
    learning_data_csv_name=str(Path().joinpath('env', 'training_data', 'output.csv')),
)

### Training and saving the Model

In [ ]:
from stable_baselines3.a2c import A2C

%load_ext tensorboard
env = environments.get_training_env()
model = A2C(
    env=env,
    policy='MlpPolicy',
    n_steps=100,
    # learning_rate=0.001,
    # learning_starts=0,
    # train_freq=1,
    # target_update_interval=500,
    # exploration_fraction=0.05,
    # exploration_final_eps=0.01,
    verbose=1,
    tensorboard_log='tensorboard'
)
model_name = 'agent'
model.learn(100_000, tb_log_name='agent')
model.save(Path().joinpath('env', 'agents', 'agent.zip'))

Giving the model a test run in an evaluation environment

In [ ]:
#63 (ns green), 86 (incoming collision), 134 (we green), 0739 unprotected good

from stable_baselines3.a2c import A2C
from pathlib import Path

env = environments.get_demonstration_env()
model = A2C(env=env, policy='MlpPolicy').load(
    Path().joinpath('env', 'agents', 'scratch_s80_f0.5.zip'))

rewards = []
actions = []
obs, info = env.reset()

gui = env.sumo.gui
gui.setBoundary(gui.DEFAULT_VIEW, 480.0, 480.0, 520.0, 520.0)
# gui.addView('3D', in3D=True)
screenshots_path_jpg = Path().joinpath('screenshots', 'tmp', 'jpg')
screenshots_path_jpg.mkdir(parents=True, exist_ok=True)
screenshots_path_svg = Path().joinpath('screenshots', 'tmp', 'svg')
screenshots_path_svg.mkdir(parents=True, exist_ok=True)

#vehicletype = env.sumo.vehicletype
#vehicletype.setMaxSpeed('carCustom', SPEED)
#vehicletype.setDecel('carCustom', vehicletype.getDecel('carCustom') * FRICTION)
#vehicletype.setEmergencyDecel('carCustom',
#                              vehicletype.getEmergencyDecel('carCustom') * FRICTION)
#for traffic_signal in env.traffic_signals.values():
#    for lane in traffic_signal.lanes:
#        traffic_signal.sumo.lane.setParameter(lane, 'frictionCoefficient', FRICTION)

done = False
while not done:
    action, _state = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    rewards.append(reward)
    actions.append(action)
    done = terminated or truncated
    current_step = int(env.sumo.simulation.getTime())
    screenshot_path_jpg = screenshots_path_jpg.joinpath(str(current_step).zfill(4) + '.jpg')
    screenshot_path_svg = screenshots_path_svg.joinpath(str(current_step).zfill(4) + '.svg')
    gui.screenshot(gui.DEFAULT_VIEW, str(screenshot_path_jpg))
    gui.screenshot(gui.DEFAULT_VIEW, str(screenshot_path_svg))
env.close()

print(rewards)
print(" ")
print(actions)

### Produce Traces
Run the simulation repeatably to produce traces (data) for causal discovery.


**Desired Output: One CSV File containing all interesting data**


In [ ]:
# Editable Parameters in 'env/generate.py'
! python env/generate.py

#### Data Summary

In [ ]:
from statistics import mean
import glob
from pathlib import Path
import xml.etree.ElementTree as ElementTree
import pandas as pd

data_folder = Path().joinpath('traces')
experiments = glob.glob(str(data_folder.joinpath('*')))

for experiment in experiments:
    experiment_path = Path(experiment)
    statistics_files = glob.glob(str(experiment_path.joinpath('*_statistics.xml')))
    ids = [path.split('/')[-1].split('_')[0] for path in statistics_files]

    data = []
    for id in ids:
        statistics_file = experiment_path.joinpath(id + '_statistics.xml')
        collisions_file = experiment_path.joinpath(id + '_collisions.xml')
        ssm_file = experiment_path.joinpath(id + '_ssm.xml')
        metadata_file = experiment_path.joinpath(id + '_metadata.xml')
        tripinfo_file = experiment_path.joinpath(id + '_tripinfo.xml')
        statistics_xml = ElementTree.parse(statistics_file).getroot()
        collisions_xml = ElementTree.parse(collisions_file).getroot()
        ssm_xml = ElementTree.parse(ssm_file).getroot()
        metadata_xml = ElementTree.parse(metadata_file).getroot()
        tripinfo_xml = ElementTree.parse(tripinfo_file).getroot()

        row = {
            'experiment': experiment,
            'index': int(id),
            'desiredSpeed': float(metadata_xml.find('.//desiredSpeed').text),
            'friction': float(metadata_xml.find('.//friction').text)
        }

        for key, value in {**statistics_xml.find('vehicleTripStatistics').attrib,
                           **statistics_xml.find('safety').attrib}.items():
            match key:
                case 'count' | 'emergencyStops' | 'emergencyBraking':
                    row[key] = int(value)
                case 'collisions':
                    row[key] = int(value)
                    row['rearEndCollisions'] = sum(
                        'southEast' in child.attrib.get('victim') for child in collisions_xml)
                    row['lateralCollisions'] = sum(
                        'southEast' in child.attrib.get('collider') for child in collisions_xml)
                    row[key] = row['rearEndCollisions'] + row['lateralCollisions']
                case _:
                    row[key] = float(value)

        waiting_times = [float(tripinfo.attrib.get('waitingTime')) for tripinfo in tripinfo_xml.findall('tripinfo')]
        average_waiting_time = mean(waiting_times)
        row['waitingTime'] = average_waiting_time

        data.append(row)

        df = pd.DataFrame(data)
        df.to_csv(experiment_path.joinpath('.summary.csv'), index=False)

Some Visualization

In [ ]:
import pandas as pd
from pathlib import Path
from matplotlib import pyplot as plt
import seaborn as sns

data = Path().joinpath('traces', 'traces_all_agents.csv')
charts_path = Path().joinpath('charts')
df = pd.read_csv(data)
'''
sns.displot(data=df, x='friction', y='waitingTime', kind='kde')
plt.title('friction - waitingTime, all desiredSpeeds')
# plt.savefig(charts_path.joinpath('friction-waitingTime.png'), bbox_inches='tight')

sns.displot(data=df, x='friction', y='collisions', kind='kde')
plt.title('friction - collisions, all desiredSpeeds')
# plt.savefig(charts_path.joinpath('friction-collisions'), bbox_inches='tight')

sns.displot(data=df, x='desiredSpeed', y='waitingTime', kind='kde')
plt.title('desiredSpeed - waitingTime, all frictions')
# plt.savefig(charts_path.joinpath('desiredSpeed-waitingTime'), bbox_inches='tight')

sns.displot(data=df, x='desiredSpeed', y='collisions', kind='kde')
plt.title('desiredSpeed - collisions, all frictions')
# plt.savefig(charts_path.joinpath('desiredSpeed-collisions'), bbox_inches='tight')

sns.displot(data=df, x='friction', y='emergencyBraking', kind='kde')
plt.title('friction - emergencyBraking, all desiredSpeeds')
# plt.savefig(charts_path.joinpath('friction-emergencyBraking'), bbox_inches='tight')

sns.displot(data=df, x='friction', y='speed', kind='kde')
plt.title('friction - speed, all desiredSpeeds')
# plt.savefig(charts_path.joinpath('friction-speed'), bbox_inches='tight')

sns.displot(data=df, x='desiredSpeed', y='emergencyBraking', kind='kde')
plt.title('desiredSpeed - emergencyBraking, all frictions')
# plt.savefig(charts_path.joinpath('desiredSpeed-emergencyBraking'), bbox_inches='tight')

sns.displot(data=df, x='desiredSpeed', y='speed', kind='kde')
plt.title('desiredSpeed - speed, all frictions')
# plt.savefig(charts_path.joinpath('desiredSpeed-speed'), bbox_inches='tight')
'''
sns.regplot(data=df[(df['friction'] >= 0.4) & (df['friction'] <= 0.6) & (df['agent'] == 'scratch_s80_f0.5')],
            x='desiredSpeed', y='waitingTime', scatter=True, line_kws={"color": "red"})
plt.title('desiredSpeed - waitingTime, 0.5 friction')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set up the plot
plt.figure(figsize=(10, 6))

# Define friction ranges (you can adjust these)
ranges = [(0.1, 0.3), (0.3, 0.5), (0.5, 0.7), (0.7, 0.9)]
colors = ['blue', 'green', 'orange', 'purple']

# Loop through ranges and plot each
for (low, high), color in zip(ranges, colors):
    subset = df[
        (df['friction'] >= low) &
        (df['friction'] < high) &
        (df['agent'] == 'scratch_s80_f0.5')
        ]

    sns.regplot(
        data=subset,
        x='desiredSpeed',
        y='waitingTime',
        scatter=False,
        label=f'{low}-{high}',
        line_kws={'color': color}
    )

# Add legend and labels
plt.legend(title='Friction Ranges')
plt.title('Waiting Time vs Desired Speed by Friction Ranges')
plt.xlabel('Desired Speed')
plt.ylabel('Waiting Time')
plt.tight_layout()

plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set up the plot
plt.figure(figsize=(10, 6))

# Define friction ranges (you can adjust these)
ranges = [(0.1, 0.3), (0.3, 0.5), (0.5, 0.7), (0.7, 0.9)]
colors = ['blue', 'green', 'orange', 'purple']

# Loop through ranges and plot each
for (low, high), color in zip(ranges, colors):
    subset = df[
        (df['friction'] >= low) &
        (df['friction'] < high) &
        (df['agent'] == 'scratch_s80_f0.5')
        ]

    sns.regplot(
        data=subset,
        x='desiredSpeed',
        y='collisions',
        scatter=False,
        label=f'{low}-{high}',
        line_kws={'color': color}
    )

# Add legend and labels
plt.legend(title='Friction Ranges')
plt.title('Collisions vs Desired Speed by Friction Ranges')
plt.xlabel('Desired Speed')
plt.ylabel('Collisions')
plt.tight_layout()

plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


data_csv = Path().joinpath('traces', 'traces_all_agents.csv')
df = pd.read_csv(data_csv)[['agent', 'desiredSpeed', 'friction', 'speed', 'waitingTime', 'emergencyBraking', 'collisions']]

# Set up the plot
plt.figure(figsize=(10, 6))

# Define friction ranges (you can adjust these)
ranges = list(df['desiredSpeed'].unique())
colors = ['blue', 'green', 'orange', 'purple']

# Loop through ranges and plot each
for current_speed, color in zip(ranges, colors):
    subset = df[
        (df['desiredSpeed'] == current_speed) &
        (df['agent'] == 'scratch_s80_f0.5')
        ]

    sns.regplot(
        data=subset,
        x='friction',
        y='speed',
        scatter=False,
        label=f'{current_speed}',
        line_kws={'color': color}
    )

# Add legend and labels
plt.legend(title='Friction Ranges')
plt.title('waitingTime vs Friction by Friction Ranges')
plt.xlabel('Friction')
plt.ylabel('speed')
plt.tight_layout()

plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set up the plot
plt.figure(figsize=(10, 6))

# Define friction ranges (you can adjust these)
ranges = list(df['desiredSpeed'].unique())
colors = ['blue', 'green', 'orange', 'purple']

# Loop through ranges and plot each
for current_speed, color in zip(ranges, colors):
    subset = df[
        (df['desiredSpeed'] == current_speed) &
        (df['agent'] == 'scratch_s80_f0.5')
        ]

    sns.regplot(
        data=subset,
        x='friction',
        y='collisions',
        scatter=False,
        label=f'{current_speed}',
        line_kws={'color': color}
    )

# Add legend and labels
plt.legend(title='Friction Ranges')
plt.title('Collisions vs Friction by Friction Ranges')
plt.xlabel('Friction')
plt.ylabel('Collisions')
plt.tight_layout()

plt.show()


Concatenate Summary CSVs

In [ ]:
from pathlib import Path
import glob
import pandas as pd

data_folder = Path().joinpath('traces_paper')
summaries = glob.glob(str(data_folder.joinpath('*', '.summary.csv')))

dfs = []
for summary in summaries:
    agent = Path(summary).parent.name[0:-7]
    df = pd.read_csv(summary)
    df['agent'] = agent
    dfs.append(df)

final_df = pd.concat(dfs)[
    ['agent', 'desiredSpeed', 'friction', 'speed', 'waitingTime', 'emergencyBraking', 'collisions']]

final_df.to_csv(data_folder.joinpath('data.csv'), index=False)

### Causal Discovery
Discover causal graph

**Desired Output: Causal Graph XML File**

In [ ]:
from castle.common.priori_knowledge import PrioriKnowledge
import networkx as nx
import pandas as pd
import itertools
from castle.algorithms import PC, DirectLiNGAM
from pathlib import Path

data_csv = Path().joinpath('traces', 'traces_all_agents.csv')
df = pd.read_csv(data_csv)[['desiredSpeed', 'friction', 'speed', 'waitingTime', 'emergencyBraking', 'collisions']]

columns = dict(enumerate(df.columns))
column_indexes = {value: int(key) for key, value in columns.items()}

priori_knowledge = PrioriKnowledge(len(columns))
independent_variables = ['desiredSpeed', 'friction']
outcome_variables = ['waitingTime', 'collisions']
forbidden_edges = [
    *({'source': column, 'target': 'desiredSpeed'} for column in column_indexes.keys() if column != 'desiredSpeed'),
    *({'source': column, 'target': 'friction'} for column in column_indexes.keys() if column != 'friction'),
    *({'source': source_node, 'target': target_node} for source_node, target_node in
      itertools.permutations(outcome_variables, 2))
]
priori_knowledge.add_forbidden_edges(
    [(column_indexes[edge['source']], column_indexes[edge['target']]) for edge in forbidden_edges])

pc = PC(variant='stable', priori_knowledge=priori_knowledge)
pc.learn(df.values.tolist())

G_PC = nx.DiGraph(pc.causal_matrix)
H_PC = nx.relabel_nodes(G_PC, dict(enumerate(df.columns)))

color_map = [
    'green' if node in independent_variables else
    'red' if H_PC.out_degree(node) == 0 else
    'yellow' for node in H_PC.nodes
]

nx.draw(G=H_PC, node_color=color_map, node_size=1200, arrowsize=30, with_labels=True,
        pos=nx.circular_layout(H_PC))


Compute directionality of arrows

# TODO: Create Pseudo-Code and paste to expose

In [ ]:
from matplotlib import pyplot as plt
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression
import seaborn as sns

direction_adjusted_edge_confidence = {}

for edge in H_PC.edges():

    X = edge[0]
    Y = edge[1]
    assert H_PC.has_edge(X, Y)

    X_data = df[X].values.reshape(-1, 1)
    Y_data = df[Y].values.reshape(-1, 1)

    # X = f(Y)
    model_X_Y = LinearRegression()
    model_X_Y.fit(Y_data, X_data)
    X_pred = model_X_Y.predict(Y_data)
    residuals_X = X_data - X_pred

    # Y = g(X)
    model_Y_X = LinearRegression()
    model_Y_X.fit(X_data, Y_data)
    Y_pred = model_Y_X.predict(X_data)
    residuals_Y = Y_data - Y_pred

    #Spearman's rank correlation
    corr_X_resid, p_X_resid = spearmanr(X_data.ravel(), residuals_Y.ravel())
    corr_Y_resid, p_Y_resid = spearmanr(Y_data.ravel(), residuals_X.ravel())
    print(f'corr_{X}_resid:', corr_X_resid)
    print(f'corr_{Y}_resid:', corr_Y_resid)

    confidence_abs = abs(abs(corr_Y_resid) - abs(corr_X_resid))
    if abs(corr_X_resid) < abs(corr_Y_resid):
        print(f"{X} -> {Y}")
        direction_adjusted_edge_confidence[(X, Y)] = confidence_abs
    else:
        print(f"{Y} -> {X}")
        direction_adjusted_edge_confidence[(Y, X)] = confidence_abs

    print("==============================")

direction_adjusted_edges = []
for edge, confidence in direction_adjusted_edge_confidence.items():
    if edge not in H_PC.edges():
        if confidence < 0.1 or edge[1] in independent_variables:
            edge = edge[::-1]
    direction_adjusted_edges.append(edge)

I_PC = H_PC.copy()
I_PC.clear_edges()

I_PC.add_edges_from(direction_adjusted_edges)
print(direction_adjusted_edge_confidence)

color_map = [
    'green' if node in independent_variables else
    'red' if I_PC.out_degree(node) == 0 else
    'yellow' for node in H_PC.nodes
]

nx.draw(G=I_PC, node_color=color_map, node_size=1200, arrowsize=30, with_labels=True,
        pos=nx.circular_layout(H_PC))

### Automatically find model from Causal Graph

In [ ]:
causal_graph = I_PC.copy()
assert nx.is_directed_acyclic_graph(causal_graph)

node_layers = {node: 0 for node in causal_graph.nodes}
predecessors = {node: list(causal_graph.predecessors(node)) for node in causal_graph.nodes}
visited = set(node for node, predecessor_nodes in predecessors.items() if len(predecessor_nodes) == 0)

# Find layers for each node
current_layer = max(node_layers.values()) + 1
while len(visited) < len(causal_graph.nodes):
    unprocessed = set(causal_graph.nodes) - visited
    current_nodes = [node for node in unprocessed if visited.issuperset(predecessors[node])]
    for node in current_nodes:
        node_layers[node] = current_layer
    visited.update(current_nodes)
    current_layer += 1


# TODO: Pseudo Code

In [ ]:
import pytensor
from pathlib import Path
import pandas as pd
import pymc as pm
import arviz as az

pytensor.config.cxx = ''
pytensor.config.floatX = "float64"

data_csv = Path().joinpath('traces', 'traces_all_agents.csv')
df = pd.read_csv(data_csv)[['desiredSpeed', 'friction', 'speed', 'waitingTime', 'emergencyBraking', 'collisions']]


df_standardized = (df - df.mean()) / df.std()
df_standardized = df_standardized
values = {column: df_standardized[column].values for column in df_standardized.columns}

alpha = {}
beta = {}
sigma = {}
mu = {}
obs = {}
independent_data = {}

structural_causal_model = pm.Model()

with structural_causal_model:
    for node in sorted(node_layers.keys(), key=node_layers.__getitem__):
        if predecessors[node]:
            alpha[node] = pm.Normal(f"{node} alpha", mu=0, sigma=10)

            current_predecessor_string = ', '.join(f'{index}:{name}' for index, name in enumerate(predecessors[node]))
            current_node_beta_name = f"{node} beta * ({current_predecessor_string})"

            beta[node] = pm.Normal(current_node_beta_name, mu=0, sigma=10, shape=len(predecessors[node]))
            sigma[node] = pm.HalfNormal(f"{node} sigma", sigma=1)
            # maybe try exponential in the future?

            mu[node] = alpha[node]
            for i, predecessor in enumerate(predecessors[node]):
                predictor = independent_data[predecessor] if node_layers[predecessor] == 0 else obs[predecessor]
                mu[node] += beta[node][i] * predictor

            obs[node] = pm.Normal(f"{node} obs", mu=mu[node], sigma=sigma[node], observed=values[node])
        else:
            # Make sure independent variables are modeled
            independent_data[node] = pm.Data(f"{node}_data", values[node])

    #idata = pm.sample()
    #idata.to_netcdf('./scm_full_new.netcdf')

    idata = az.InferenceData.from_netcdf('./scm_full.netcdf')

az.summary(idata, round_to=2)

SCM Direct Effect:

In [ ]:
import pytensor
from pathlib import Path
import pandas as pd
import pymc as pm
import arviz as az

pytensor.config.cxx = ''
pytensor.config.floatX = "float64"

data_csv = Path().joinpath('traces', 'traces_all_agents.csv')
df = pd.read_csv(data_csv)[['desiredSpeed', 'friction', 'speed', 'waitingTime', 'emergencyBraking', 'collisions']]
df_standardized = (df - df.mean()) / df.std()
df_standardized = df_standardized
values = {column: df_standardized[column].values for column in df_standardized.columns}

outcome_variables = [node for node in I_PC.nodes if I_PC.out_degree(node) == 0]

alpha = {}
beta = {}
sigma = {}
mu = {}
obs = {}
independent_data = {}

scm_direct_effect = pm.Model()

with scm_direct_effect:
    for node in sorted(node_layers.keys(), key=node_layers.__getitem__):
        if node in outcome_variables:
            alpha[node] = pm.Normal(f"{node} alpha", mu=0, sigma=10)

            current_predecessor_string = ', '.join(f'{index}:{name}' for index, name in enumerate(predecessors[node]))
            current_node_beta_name = f"{node} beta * ({current_predecessor_string})"

            beta[node] = pm.Normal(current_node_beta_name, mu=0, sigma=10, shape=len(predecessors[node]))
            sigma[node] = pm.HalfNormal(f"{node} sigma", sigma=1)

            mu[node] = alpha[node]
            for i, predecessor in enumerate(predecessors[node]):
                predictor = independent_data[predecessor]
                mu[node] += beta[node][i] * predictor

            obs[node] = pm.Normal(f"{node} obs", mu=mu[node], sigma=sigma[node], observed=values[node])
        else:
            # Make sure independent variables are modeled
            independent_data[node] = pm.Data(f"{node}_data", values[node])

    idata = pm.sample(draws=100, tune=100)
    idata.to_netcdf(str(Path().joinpath('structural_causal_models', 'scm_direct_effect.netcdf')))

    # idata = az.InferenceData.from_netcdf('./scm_full.netcdf')

az.summary(idata, round_to=2)

In [ ]:
import matplotlib.pyplot as plt

model_graph = pm.model_to_networkx(scm_direct_effect)

plt.figure(figsize=(20, 20))  # Set viewport size (width, height in inches)

nx.draw(
    model_graph,
    pos=nx.planar_layout(model_graph, scale=5),  # Increase node spacing
    with_labels=True,
    node_size=5000,  # Increase node size (default 300)
    node_color='skyblue',  # Better visibility
    edge_color='gray',  # Better visibility
    width=3,  # Increase edge thickness (default 1)
    font_size=14,  # Increase label size
    font_weight='bold',  # Improve label readability
    arrowsize=30  # Increase arrow size for directed edges
)

plt.show()

### Generate Posterior Distribution

In [ ]:
FIX_DESIRED_SPEED = True
FIX_FRICTION = True
DESIRED_SPEED = 13.89
FRICTION = 0.75

desired_speed = (DESIRED_SPEED - df['desiredSpeed'].mean()) / df['desiredSpeed'].std()
desired_speed = [desired_speed] * len(df_standardized)

friction = (FRICTION - df['friction'].mean()) / df['friction'].std()
friction = [friction] * len(df_standardized)

idata = az.InferenceData.from_netcdf('./scm_full.netcdf')
thinned_idata = idata.sel(draw=slice(None, None, len(df_standardized)))

with structural_causal_model:
    if FIX_DESIRED_SPEED:
        pm.set_data({'desiredSpeed_data': desired_speed})
    if FIX_FRICTION:
        pm.set_data({'friction_data': friction})
    predictions = pm.sample_posterior_predictive(thinned_idata, predictions=True, var_names=["collisions obs"])

### Visualization
prediction_data = predictions.predictions
prediction_df = prediction_data.to_dataframe()

collision_obs = prediction_df['collisions obs'] * df['collisions'].std() + df['collisions'].mean()

df_filtered = df.copy()
if FIX_DESIRED_SPEED:
    df_filtered = df_filtered[(df_filtered['desiredSpeed'] == DESIRED_SPEED)]
if FIX_FRICTION:
    df_filtered = df_filtered[(df_filtered['friction'] >= FRICTION - 0.05) & (df_filtered['friction'] <= FRICTION + 0.05)]

# Sample matching number of predictions
sampled_predictions = collision_obs.sample(n=len(df_filtered)).reset_index(drop=True)

# Create figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# KDE Plot Comparison
sns.kdeplot(collision_obs, label='Predicted', fill=True, ax=ax1)
sns.kdeplot(df_filtered['collisions'], label='Observed', fill=True, ax=ax1)
ax1.set_title('Density Comparison')
ax1.set_xlabel('Collisions')
ax1.legend()

# Boxplot Comparison
boxplot_data = {
    'Predicted': sampled_predictions,
    'Observed': df_filtered['collisions'].reset_index(drop=True)
}
sns.boxplot(data=pd.DataFrame(boxplot_data), ax=ax2)
ax2.set_title('Distribution Spread')
ax2.set_ylabel('Collisions')

plt.tight_layout()
plt.show()

### Fit Weibull

In [ ]:
weibull_model = pm.Model()

data_csv = Path().joinpath('distribution_shift', 'traces', 'scratch_s80_f1_shift_s80_f0.8.xml', '.summary.csv')
df_weibull = pd.read_csv(data_csv)[['collisions']]

with weibull_model:
    weibull_alpha = pm.HalfNormal('weibull_alpha', sigma=1)  # Shape parameter
    weibull_beta = pm.HalfNormal('weibull_beta', sigma=1)    # Scale parameter
    weibull_collision_obs = pm.Weibull("collisions", alpha=weibull_alpha, beta=weibull_beta, observed=df_weibull['collisions'])
    weibull_idata = pm.sample()

az.summary(weibull_idata, round_to=2)

with weibull_model:
    predictions = pm.sample_posterior_predictive(weibull_idata, predictions=True)

predictions

In [ ]:
prediction_data = predictions.predictions
prediction_df = prediction_data.to_dataframe()

sns.histplot(data=prediction_df, x='collisions', bins=50)
plt.title('Distribution of Collision Column')
plt.xlabel('Collision')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()